<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/clip_cosine_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Endovis 2018

In [1]:
import gdown

#endovis18 dataset
url = 'https://drive.google.com/uc?id=1lRNAgC-6QgIQd-vum-jr523tYPr6yNM7'
gdown.download(url,'endovis18.zip',quiet=True)
!unzip -q endovis18.zip

url = 'https://drive.google.com/uc?id=1hY6jXjREiDXGx25bSdzNYb58amYkTzCL'
gdown.download(url,'paper_fig.zip',quiet=True)
!unzip -q paper_fig.zip

Using OpenCLIP

In [5]:
!pip -q install git+https://github.com/openai/CLIP.git
!pip -q install open_clip_torch
!pip -q install sentence_transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 961.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [15]:
import torch
import open_clip
import cv2
from sentence_transformers import util
from PIL import Image
import cv2
from glob import glob
import numpy as np

# Load images
img1_path = '/content/paper_fig/img1.png'
img2_path= '/content/endovis18/seq_1/left_frames/frame000.png'

device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
def imageEncoder(img):
    img1 = Image.fromarray(img).convert('RGB')
    img1 = preprocess(img1).unsqueeze(0).to(device)
    img1 = model.encode_image(img1)
    return img1
def generateScore(image1, image2):
    test_img = cv2.imread(img1_path, cv2.IMREAD_UNCHANGED)
    data_img = cv2.imread(img2_path, cv2.IMREAD_UNCHANGED)
    img1 = imageEncoder(test_img)
    img2 = imageEncoder(data_img)
    cos_scores = util.pytorch_cos_sim(img1, img2)
    score = round(float(cos_scores[0][0])*100, 2)
    return score


print(f"similarity Score: ", round(generateScore(img_path, img_path), 2))

similarity Score:  86.3


In [ ]:
from torch.utils.data import Dataset, DataLoader
from glob import glob
import numpy as np
from sentence_transformers import util

class ImageDataset(Dataset):
    def __init__(self, root_paths =None, transform=None):
        self.img_path_all = []
        for root_path in root_paths:
            self.img_path_all.extend(glob (root_path))
        self.transform = transform

    def __len__(self):
        return len(self.img_path_all)

    def __getitem__(self, i):
        img = Image.open(self.img_path_all[i]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.img_path_all[i]

# def main():


device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16-plus-240', pretrained="laion400m_e32")
model.to(device)
root_paths = ['endovis18/*/left_frames/*.png', 'endovis17/*/images/*.png']
paper_root_paths = ['/content/paper_fig/*.png']
img_dataset = ImageDataset(root_paths=root_paths, transform=preprocess)
paper_img_dataset = ImageDataset(root_paths=paper_root_paths, transform=preprocess)
paper_imgs, paper_paths = paper_img_dataset[0]
img_batch, paths = img_dataset[0]
paper_img_features = model.encode_image(paper_imgs[None].to(device))
feature_batch = model.encode_image(img_batch[None].to(device))


In [14]:
paper_img_features = model.encode_image(paper_imgs[None].to(device))
feature_batch = model.encode_image(img_batch[None].to(device))
cos_scores = util.pytorch_cos_sim(feature_batch, paper_img_features)
cos_scores

tensor([[0.7323]], device='cuda:0', grad_fn=<MmBackward0>)